In [2]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

In [3]:
listings = pd.concat([pd.read_csv('data/nyc_listings_outdoorspace_0.csv'), pd.read_csv('data/nyc_listings_outdoorspace_1.csv'), pd.read_csv('data/nyc_listings_outdoorspace_2.csv')])
ocr_results = pd.read_csv('data/ocr_results.csv')
listings.head()

,id,address,unit_number,description,property_type_code,balcony
0,1,111 Central Park North,3A,This Modern Masterpiece offers over 75 feet of...,540,False
1,45,508 West 24th Street,2N,"Adjacent to the High Line, 508 West 24th Stree...",540,True
2,46,508 West 24th Street,PHS,"MOVE IN NOW!! Penthouse South is a gorgeous, s...",540,NaN
3,55,130 Barrow Street,PH507,Amazing West Village Penthouse duplex apartmen...,540,True
4,58,166 West 18th Street,PH12,Rare full-floor condo with exceptional enterta...,540,NaN


In [4]:
ocr_results.columns = ['listing_id', 'public_or_private']
ocr_results['decision_source'] = ['ocr'] * len(ocr_results)
ocr_results.head()

,listing_id,public_or_private,decision_source
0,279,private,ocr
1,393,public,ocr
2,441,private,ocr
3,560,public,ocr
4,567,public,ocr


In [5]:
results = listings.merge(ocr_results, how='outer', left_on='id', right_on='listing_id')
results.head()

,id,address,unit_number,description,property_type_code,balcony,listing_id,public_or_private,decision_source
0,1,111 Central Park North,3A,This Modern Masterpiece offers over 75 feet of...,540,False,NaN,NaN,NaN
1,45,508 West 24th Street,2N,"Adjacent to the High Line, 508 West 24th Stree...",540,True,NaN,NaN,NaN
2,46,508 West 24th Street,PHS,"MOVE IN NOW!! Penthouse South is a gorgeous, s...",540,NaN,NaN,NaN,NaN
3,55,130 Barrow Street,PH507,Amazing West Village Penthouse duplex apartmen...,540,True,NaN,NaN,NaN
4,58,166 West 18th Street,PH12,Rare full-floor condo with exceptional enterta...,540,NaN,NaN,NaN,NaN


In [6]:
# TODO: add these
#when lower(l.description) like '%apartment features%' then 'maybe private'
#when lower(l.description) like '%unit features%' then 'maybe private'
#if property_type_code in [560, 580, 590, 600, 610]:
        #return True
#if a.balcony:
 #       return True

In [7]:
def private_or_public(s: str) -> bool:
    s = s.lower()
    if 'balcony' in s:
        return True
    elif 'balconies' in s:
        return True
    elif 'private terrace' in s:
        return True
    elif 'private roof' in s:
        return True
    elif 'private outdoor' in s:
        return True
    elif 'private patio' in s:
        return True
    elif 'wrap terrace' in s:
        return True
    elif 'private' in s and 'yard' in s:
        return True
    elif 'balconies' in s:
        return True
    else:
        return False

In [8]:
count = 0
for index, row in tqdm(results.iterrows(), total=len(results)):
    if not (pd.isna(row['balcony']) or row['balcony'] == False):
        results.loc[index, 'public_or_private'] = 'private'
        results.loc[index, 'decision_source'] = 'balcony_field'
    elif row['property_type_code'] in [560, 580, 590, 600, 610]:
        results.loc[index, 'public_or_private'] = 'private'
        results.loc[index, 'decision_source'] = 'property_type_field'
    elif private_or_public(str(row['description'])):
        results.loc[index, 'public_or_private'] = 'private'
        results.loc[index, 'decision_source'] = 'lising_description'
    elif row['public_or_private'] == 'private':
        continue
    else:
        count += 1
        results.loc[index, 'public_or_private'] = 'public'
        results.loc[index, 'decision_source'] = np.nan

results.head()


  0%|          | 0/328377 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
print(count)

81784


In [ ]:
results.groupby('decision_source')['id'].nunique()

decision_source
balcony_field          140874
lising_description      54750
ocr                      4366
property_type_field     46603
Name: id, dtype: int64

In [ ]:
4366/len(results)

0.013295693669166843

In [ ]:
len(results[results['public_or_private'] != 'private'])

81784

In [ ]:
results.to_csv('data/outdoor_space_results.csv')

In [9]:
results = pd.read_csv('data/outdoor_space_results.csv')
len(results)

328377